In [1]:
import numpy as np
import pandas as pd
import seaborn as sns

In [2]:
dataset = pd.read_csv('Churn_Modelling.csv')

In [3]:
dataset.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [4]:
dataset.isna().sum()

RowNumber          0
CustomerId         0
Surname            0
CreditScore        0
Geography          0
Gender             0
Age                0
Tenure             0
Balance            0
NumOfProducts      0
HasCrCard          0
IsActiveMember     0
EstimatedSalary    0
Exited             0
dtype: int64

In [5]:
X = dataset.iloc[: , 3:13].values
y=dataset.iloc[: , -1].values

In [6]:
X[0]

array([619, 'France', 'Female', 42, 2, 0.0, 1, 1, 1, 101348.88],
      dtype=object)

In [7]:
from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()
X[: , 2] = labelencoder.fit_transform(X[: , 2])

In [8]:
X[0]

array([619, 'France', 0, 42, 2, 0.0, 1, 1, 1, 101348.88], dtype=object)

In [10]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers = [('encoder' , OneHotEncoder() , [1])] , remainder='passthrough')
X = np.array(ct.fit_transform(X))

In [11]:
X[0]

array([1.0, 0.0, 0.0, 619, 0, 42, 2, 0.0, 1, 1, 1, 101348.88],
      dtype=object)

In [12]:
X = X[: ,1:] #to avoid dummy variable trap

In [13]:
X[0]

array([0.0, 0.0, 619, 0, 42, 2, 0.0, 1, 1, 1, 101348.88], dtype=object)

In [14]:
from sklearn.model_selection import train_test_split
x_train , x_test , y_train , y_test = train_test_split(X , y, test_size=0.2 , random_state=0)

In [15]:
from xgboost import XGBClassifier
classifier = XGBClassifier()
classifier.fit(x_train , y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [18]:
y_pred = classifier.predict(x_test)

In [19]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test , y_pred)

0.8645

In [20]:
from sklearn.model_selection import GridSearchCV

In [21]:
parameters = {
    'learning_rate':[0.1,0.15,0.2,0.25,0.3],
    'max_depth':[3,4,5,6,7],
    'gamma':[0.0 , 0.1,0.2,0.3, 0.4],
    'min_child_width':[1,2,3,4,5,6]
}

In [22]:
gridsearch = GridSearchCV(estimator=classifier , param_grid=parameters , scoring='neg_log_loss' , cv=10 , n_jobs=-1)

In [23]:
gridsearch = gridsearch.fit(x_train , y_train)

In [24]:
gridsearch.best_estimator_

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0.0, learning_rate=0.2, max_delta_step=0,
       max_depth=3, min_child_weight=1, min_child_width=1, missing=None,
       n_estimators=100, n_jobs=1, nthread=None,
       objective='binary:logistic', random_state=0, reg_alpha=0,
       reg_lambda=1, scale_pos_weight=1, seed=None, silent=True,
       subsample=1)

In [25]:
gridsearch.best_params_

{'gamma': 0.0, 'learning_rate': 0.2, 'max_depth': 3, 'min_child_width': 1}

In [26]:
gridsearch.best_score_

-0.3335100695791589

In [ ]:
#Thank you!!